In [1]:
# TensorFlow Implementation
#  TensorFlow library, a powerful open-source library developed by Google for machine learning and deep learning tasks.
import tensorflow as tf
# keras-> tensorflow api used to buid and train the neural network model
# 1)layers -> like dense layer , convolutional layer, recurrent layer
# 2)models -> Provides a structure for building and managing a neural network model in Keras.
from tensorflow.keras import layers, models 

import numpy as np

# Generate some sample data
np.random.seed(42)
X_train = np.random.rand(1000, 10)
y_train = np.sum(X_train, axis=1) > 5

# TensorFlow Model
# models.Sequential creates a sequential model where layers are stacked in order
def create_tf_model():
    model = models.Sequential([
    #his is a Dense (fully connected) layer with 64 units (neurons) and a ReLU (Rectified Linear Unit) activation function.
    #The input_shape=(10,) specifies that the model expects each input to have 10 features, matching the shape of X_train.
    #ReLU helps the network learn non-linear relationships.
        layers.Dense(64, activation='relu', input_shape=(10,)),
    # Dropout is a regularization technique where a fraction of neurons (20% in this case) are randomly "dropped" (ignored) during training
        layers.Dropout(0.2),
    # t learns more complex relationships
        layers.Dense(32, activation='relu'),
    # Since this model is set up for binary classification, the sigmoid function outputs a probability between 0 and 1
        layers.Dense(1, activation='sigmoid')
    ])
    # The optimizer is like the engine that drives the learning process. "Adam" is a specific optimizer that helps the model adjust its internal settings (called weights) efficiently to minimize errors during training. It’s popular because it works well on many types of problems.
    # The loss function is how we measure the model’s mistakes. Since this model is for a binary classification problem (predicting one of two classes, like True/False), we use "binary cross-entropy" as the loss function. This function calculates how close the model’s predictions are to the actual answers. The goal is to make this loss as small as possible, which means the model is learning well.
    # metric tell us how accurate our model predict it gives us the percentage of correct orediction
    model.compile(optimizer='adam',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    return model
# his function builds a simple feed-forward neural network for binary classification with a dropout layer to prevent overfitting and a sigmoid output layer for probability-based binary predictions. The model is compiled with settings suitable for binary classification tasks

# Train TensorFlow model
tf_model = create_tf_model()
#  epochs -> The model will go through the entire dataset 10 times
# batch_size=32: -> The data is processed in "batches" of 32 samples at a time. 
# validation_split=0.2,-> Reserves 20% of X_train and y_train for validation (checking model performance on unseen data) while training on the remaining 80%.
# verbose=0: Runs the training silently, without printing updates in the console. Setting verbose=1 will show progress for each epoch.
tf_history = tf_model.fit(X_train, y_train,
                         epochs=10,
                         batch_size=32,
                         validation_split=0.2,
                         verbose=0)

# PyTorch Implementation
# import torch:

# This imports the main PyTorch library, which provides the core functionalities for tensor operations and building neural networks.
# import torch.nn as nn:

# This imports the nn module from PyTorch, which contains classes and functions to build neural networks, including various layers, loss functions, and activation functions.
# import torch.optim as optim:

# This imports the optim module, which provides optimization algorithms (like SGD, Adam, etc.) to update the parameters of your neural network during training.
# from torch.utils.data import TensorDataset, DataLoader:

# This imports two classes:
# TensorDataset: This is a dataset class that allows you to wrap your input tensors (features) and target tensors (labels) into a single dataset object. This makes it easier to iterate over the data in batches.
# DataLoader: This is a utility that helps load your data in batches, shuffling the data if desired. It provides an iterable over the dataset, making it efficient to feed data to the model during training.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors

# This indicates that the tensor will contain 32-bit floating-point numbers. 
# After this conversion, x_torch can be used directly in PyTorch for model training or inference.
X_torch = torch.FloatTensor(X_train)
y_torch = torch.FloatTensor(y_train)

# Create dataset and dataloader
dataset = TensorDataset(X_torch, y_torch)
#  DataLoader simplifies the training loop, as you can directly iterate over batches of data in your training process, making the code cleaner and more efficient.
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# PyTorch Model
# in pytorch all neural network model are inherited from nn.Module class
# def __init__(self): -> This is the initializer (constructor) method for the class. It is called when you create an instance of PyTorchMode
# |Inside this method, you define the layers and components of your neural network.
# super(PyTorchModel, self).__init__()-> This line calls the constructor of the parent class (nn.Module). It ensures that the parent class is properly initialized, allowing your model to inherit the functionality provided by nn.Module
class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
         self.layer1 = nn.Linear(10, 64)     #It takes 10 input features (the size of the input data) and transforms them into 64 output features (neurons in this layer).
        self.layer2 = nn.Linear(64, 32)      #This is typically used for binary classification tasks, where the model outputs a probability score
        self.layer3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()     #ReLU is commonly used as an activation function in neural networks because it introduces non-linearity, allowing the model to learn complex patterns.
        self.dropout = nn.Dropout(0.2)   #keep 20% for validation
        self.sigmoid = nn.Sigmoid()   #This creates an instance of the Sigmoid activation function, which squashes the output to a range between 0 and 1.

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

# Initialize model, loss function, and optimizer
torch_model = PyTorchModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(torch_model.parameters())

# Training loop for PyTorch
def train_pytorch_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

# Train PyTorch model
train_pytorch_model(torch_model, dataloader, criterion, optimizer)

# Example predictions
def make_predictions():
    # Generate test data
    X_test = np.random.rand(5, 10)

    # TensorFlow predictions
    tf_preds = tf_model.predict(X_test)

    # PyTorch predictions
    torch_model.eval()
    with torch.no_grad():
        torch_preds = torch_model(torch.FloatTensor(X_test))

    return X_test, tf_preds, torch_preds.numpy()

# Make and display predictions
X_test, tf_preds, torch_preds = make_predictions()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step



1. What is a tensor?
A tensor is a fundamental data structure used in deep learning - it's a generalization of vectors and matrices to potentially higher dimensions. Think of it as:
- Scalar: 0-dimensional tensor (single number)
- Vector: 1-dimensional tensor (list of numbers)
- Matrix: 2-dimensional tensor (table of numbers)
- N-dimensional array: Higher dimensional tensor

Example:
```python
# Scalar (0D tensor)
scalar = tf.constant(5)

# Vector (1D tensor)
vector = tf.constant([1, 2, 3, 4])

# Matrix (2D tensor)
matrix = tf.constant([[1, 2], [3, 4]])

# 3D tensor
tensor_3d = tf.constant([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
```

2. Dimensions and Ranks in TensorFlow:
- Rank: Number of dimensions in a tensor
- Shape: Length of each dimension

Examples:
```python
# Rank 0 (scalar): shape []
t0 = tf.constant(42)

# Rank 1 (vector): shape [3]
t1 = tf.constant([1, 2, 3])

# Rank 2 (matrix): shape [2, 3]
t2 = tf.constant([[1, 2, 3],
                 [4, 5, 6]])

# Rank 3: shape [2, 2, 2]
t3 = tf.constant([[[1, 2], [3, 4]],
                 [[5, 6], [7, 8]]])
```

3. Building Models in Keras:
There are three main ways to build models in Keras:

a. Sequential API (simplest):
```python
from tensorflow.keras import Sequential, layers

model = Sequential([
    layers.Dense(64, activation='relu', input_shape=(784,)),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])
```

b. Functional API (more flexible):
```python
from tensorflow.keras import Model, Input

inputs = Input(shape=(784,))
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)
```

c. Subclassing (most flexible):
```python
class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.dense1 = layers.Dense(64, activation='relu')
        self.dropout = layers.Dropout(0.2)
        self.dense2 = layers.Dense(10, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dropout(x)
        return self.dense2(x)
```

4. Function Operations in Theano:
Theano (though now deprecated) introduced several key concepts still used in modern frameworks:
- Symbolic Variables: Placeholders for data
- Computational Graphs: Operations arranged in a directed graph
- Function Compilation: Converting symbolic expressions to efficient code

Example of Theano-style operations (modern equivalent in TensorFlow):
```python
import tensorflow as tf

# Define variables
x = tf.Variable(initial_value=3.0)
y = tf.Variable(initial_value=2.0)

# Define operation
@tf.function  # Similar to Theano's function compilation
def compute_z(x, y):
    return tf.square(x) + y

# Execute operation
z = compute_z(x, y)
```

5. PyTorch vs TensorFlow Differences:

Key Differences:
1. Dynamic vs Static Graphs:
```python
# PyTorch (Dynamic)
class PyTorchModel(nn.Module):
    def forward(self, x):
        # Can modify behavior at runtime
        if self.training:
            return x * 2
        return x

# TensorFlow (Static with @tf.function)
@tf.function
def tensorflow_model(x):
    # Graph is fixed after first run
    return x * 2
```

2. Eager Execution:
```python
# PyTorch (Always eager by default)
x = torch.tensor([1, 2, 3])
y = x + 2  # Immediate execution

# TensorFlow (Can switch between eager and graph)
tf.config.run_functions_eagerly(True)  # Enable eager mode
x = tf.constant([1, 2, 3])
y = x + 2  # Immediate execution
```

3. API Design:
```python
# PyTorch (Object-Oriented)
model = nn.Sequential(
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Linear(5, 1)
)

# TensorFlow (More functional)
model = tf.keras.Sequential([
    layers.Dense(5, activation='relu', input_shape=(10,)),
    layers.Dense(1)
])
```

The main philosophical differences are:
- PyTorch is more Python-native and research-friendly
- TensorFlow is more production-focused with better deployment tools
- PyTorch has a more imperative style
- TensorFlow has better visualization tools (TensorBoard)
- PyTorch has better debugging capabilities due to its dynamic nature

